## Importing

In [1]:
import pandas as pd
import scipy as sp
import numpy as np

import emcee
from multiprocessing import Pool

import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from matplotlib import font_manager
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from cycler import cycler
import corner
from typing import Any, Dict, List, Optional, Tuple, Union

import lymph

To do: 
Sampling mit ultimate truth consensus. 

## Creating or Loading Model

In [2]:
filename = "../data/extended_system.hdf5"

graph = {
    ('tumor', 'primary'): ['I', 'II', 'III', 'IV', 'V', 'VII'],
    ('lnl', 'I'): [],
    ('lnl', 'II'): ['I', 'III', 'V', 'VII'],
    ('lnl', 'III'): ['IV'],
    ('lnl', 'IV'): [],
    ('lnl', 'V'): [],
    ('lnl', 'VII'): [],
}
extended_systm = lymph.Unilateral(graph=graph)


print(extended_systm)

Unilateral lymphatic system with 1 tumor(s) and 6 LNL(s).
primary-0.0%->I primary-0.0%->II primary-0.0%->III primary-0.0%->IV primary-0.0%->V primary-0.0%->VII II-0.0%->I II-0.0%->III II-0.0%->V II-0.0%->VII III-0.0%->IV


## Modalities

In [3]:
spsn = {"PET": [0.86, 0.79],
        "MRI": [0.63, 0.81],
        "diagnostic_consensus": [0.63, 0.81],
        "pathology": [1., 1.]}
#                         ^   ^
#                specificty   sensitivity
extended_systm.modalities = spsn

## Data

In [4]:

data = pd.read_csv("../data/latest.csv", header=[0,1,2] )
t_stage = data.iloc[:,18]
data = data.iloc[:,21:168]
ipsi_data = data.xs("ipsi",level=1,axis=1)
t_stage.loc[t_stage <= 2, ] = "early"
t_stage.loc[t_stage!="early", ] = "late"


ipsi_data["info","t_stage"]= t_stage
ipsi_data = ipsi_data.drop(['Ia', 'Ib',"IIa","IIb"], level=1, axis=1)
ipsi_data = ipsi_data.drop(['CT', 'FNA',"pCT"], level=0, axis=1)

ipsi_data.to_csv("../data/ipsi_data.csv", index=False)

extended_systm.patient_data = ipsi_data


C:\Users\Luca\AppData\Local\Temp\ipykernel_25880\1607458719.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipsi_data["info","t_stage"]= t_stage


Loading patients of late T-stage


100%|██████████| 14336/14336 [00:12<00:00, 1132.03it/s]


Loading patients of early T-stage


100%|██████████| 20864/20864 [00:18<00:00, 1150.99it/s]


## Storage of model

In [5]:
extended_systm.to_hdf(
    filename=filename, 
    name="extended/model"
)

C:\Users\Luca\anaconda3\envs\lymph2\lib\site-packages\lymph\utils.py:268: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->MultiIndex([(                 'MRI',       'I'),
            (                 'MRI',      'II'),
            (                 'MRI',     'III'),
            (                 'MRI',      'IV'),
            (                 'MRI',       'V'),
            (                 'MRI',     'VII'),
            (                 'PET',       'I'),
            (                 'PET',      'II'),
            (                 'PET',     'III'),
            (                 'PET',      'IV'),
            (                 'PET',       'V'),
            (                 'PET',     'VII'),
            ('diagnostic_consensus',       'I'),
            ('diagnostic_consensus',      'II'),
            ('diagnostic_consensus',     'III'),
            ('di

## Likelihood

In [6]:
#settings for the binom distributions
early_p=0.3
max_t=10
t = np.arange(max_t + 1)

def llh(theta):
    spread_probs, late_p = theta[:-1], theta[-1]
    
    print("run")
    if late_p > 1. or late_p < 0.:
        return -np.inf
    
    
    time_dists={
        "early": sp.stats.binom.pmf(t, max_t, early_p),
        "late" : sp.stats.binom.pmf(t, max_t, late_p),
    }
     
    return extended_systm.marginal_log_likelihood(spread_probs, t_stages=["early", "late"], time_dists=time_dists)

## Sampling

In [11]:
# Settings for the sampler
ndim = len(extended_systm.spread_probs) + 1
nwalkers = 20 * ndim
nstep = 10000
burnin = 5000
moves = [(emcee.moves.DEMove(), 0.8), (emcee.moves.DESnookerMove(), 0.2)]


#prepare the backend
new_backend = emcee.backends.HDFBackend(
    filename=filename,
    name="extended/samples"
)
#backend.reset(nwalkers, ndim)
print("Initial size: {0}".format(new_backend.iteration))

# starting point
initial_spread_probs = np.random.uniform(low=0., high=1., size=(nwalkers,ndim))


if __name__ == "__main__":
    sampler = emcee.EnsembleSampler(
        nwalkers, ndim, 
        llh, 
        moves=moves,
        backend=new_backend
    )
    sampler.run_mcmc(initial_spread_probs, nstep-new_backend.iteration, progress=True)
    samples_HMM = sampler.get_chain(flat=True, discard=burnin)
    print(samples_HMM)      





Initial size: 5161
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


  0%|          | 0/4839 [00:00<?, ?it/s]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


  0%|          | 1/4839 [00:00<31:46,  2.54it/s]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run

...

  0%|          | 21/4839 [00:15<1:03:21,  1.27it/s]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


Traceback (most recent call last):
  File "C:\Users\Luca\anaconda3\envs\lymph2\lib\site-packages\emcee\ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "C:\Users\Luca\AppData\Local\Temp\ipykernel_25880\1399224823.py", line 15, in llh
    "early": sp.stats.binom.pmf(t, max_t, early_p),
  File "C:\Users\Luca\anaconda3\envs\lymph2\lib\site-packages\scipy\stats\_distn_infrastructure.py", line 3178, in pmf
    cond1 = (k >= _a) & (k <= _b) & self._nonzero(k, *args)
  File "C:\Users\Luca\anaconda3\envs\lymph2\lib\site-packages\scipy\stats\_distn_infrastructure.py", line 3102, in _nonzero
    return floor(k) == k
KeyboardInterrupt
  0%|          | 21/4839 [00:16<1:01:56,  1.30it/s]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
emcee: Exception while calling your likelihood function:
  params: [0.46907693 0.73040578 0.22127444 0.16114741 0.24413941 0.01446518
 0.01969153 0.5043818  0.43468537 0.08164742 1.01886673 0.22065085]
  args: []
  kwargs: {}
  exception:


KeyboardInterrupt: 